# Python에서 MySQL 사용하기

- CRUD Operation

In [2]:
import json
import pymysql

In [3]:
with open('mysql.json','r') as file:
    config_str = file.read()
config_str

'{\n    "host": "localhost",\n    "user": "hkuser",\n    "password": "hkpass",\n    "database": "world",\n    "port": 3306\n}'

In [4]:
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'hkuser',
 'password': 'hkpass',
 'database': 'world',
 'port': 3306}

In [5]:
conn = pymysql.connect(                 # connection 
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database =  config['database'],
    port =  config['port']
)

- 테이블 생성

In [6]:
sql_user = """
    CREATE TABLE if NOT EXISTS uers(
	uid VARCHAR(20) NOT NULL PRIMARY KEY,
	pwd CHAR(44),
	uname VARCHAR(20) NOT NULL,
	reg_date DATETIME DEFAULT CURRENT_TIMESTAMP,
	is_deleted INT DEFAULT 0
);
"""

In [7]:
cur = conn.cursor()
cur.execute(sql_user)

0

- 테이블 변경

In [25]:
cur.execute('alter table users add column email varchar(40);')
# email 이미 있는 상태이기때문에 에러나 감

OperationalError: (1060, "Duplicate column name 'email'")

- 데이터 입력

In [9]:
cur = conn.cursor()
cur.execute("INSERT INTO users(uid,uname) VALUES('admin','관리자');")     # 파이썬에서는  Mysql ""을 사용 / 하이디sql에서는 '' 사용

1

In [10]:
cur.execute('select * from users;')
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', None, '관리자', datetime.datetime(2021, 6, 9, 10, 53, 44), 0, None)


In [11]:
# 입력한 데이터를 즉시 flush하려면 conn.commit() 추가
sql_insert_multi = """
    INSERT INTO users(uid,uname)
        VALUES('eskim','김은숙'),('wjlee','이우정');
"""
cur.execute(sql_insert_multi)
conn.commit()           # 플러싱 해주는 역할 

In [13]:
# reg_date는 저걸만든 현재 시간!

sql_select = """
    SELECT uid,uname,DATE_FORMAT(reg_date,'%Y-%m-%d %h:%i') AS reg_date,
  is_deleted, email FROM users;
"""
cur.execute(sql_select)
rows = cur.fetchall()
for row in rows:
    print(row)

('admin', '관리자', '2021-06-09 10:53', 0, None)
('eskim', '김은숙', '2021-06-09 10:55', 0, None)
('wjlee', '이우정', '2021-06-09 10:55', 0, None)


In [16]:
# 파라메터 이용, placeholder
uid = 'djy'
uname = '대조영'
cur = conn.cursor()
sql = "INSERT INTO users(uid,uname) VALUES(%s,%s);"   # (%s,%s) 가 플레이스홀더
cur.execute(sql, (uid,uname))   # 플레이스홀더하고 여기에 데이터입력
conn.commit()   # 한줄단위로 데이터를 입력해서 출력한다

In [17]:
# uid 하나만 있는경우
uid = 'gdhong'
cur = conn.cursor()
sql = "INSERT INTO users(uid,uname) VALUES(%s, '홍길동');"
cur.execute(sql, (uid,))  # 튜플이기때문에 ()안에 uid를 넣어줘야한다,,,,
conn.commit()  

In [20]:
users = (('jbpark','박재범'),('vincenzo','빈센조'))
cur = conn.cursor()
sql = "INSERT INTO users(uid,uname) VALUES(%s, %s);"
# 아래 for문은 cur.executemany(sql,user)로 대체 가능하지만 for문 사용을 추천!
for user in users:
    cur.execute(sql, user)
conn.commit() 

- 데이터 삭제

In [21]:
cur = conn.cursor()
sql = "DELETE FROM users WHERE uid != 'admin';"
cur.execute(sql)
conn.commit()  

- 판다스로 데이터프레임 만들기

In [22]:
import pandas as pd
uids = ['djy','eskim','gdhong','jbpark','vincenzo','wjlee']
unames = ['대조영','김은숙','홍길동','박재범','빈센조','이우정']
df = pd.DataFrame({'uid':uids, 'uname':unames})
df

,uid,uname
0,djy,대조영
1,eskim,김은숙
2,gdhong,홍길동
3,jbpark,박재범
4,vincenzo,빈센조
5,wjlee,이우정


In [23]:
cur = conn.cursor()
sql = "INSERT INTO users(uid,uname) VALUES(%s, %s);"
for i in df.index:
    cur.execute(sql, (df.uid[i], df.uname[i]))
conn.commit() 

- 데이터 수정

In [29]:
# 관리자의 이메일 주소 바꾸기
email = 'admin@korea.com'
uid = 'admin'
cur = conn.cursor()
# sql = "UPDATE users SET email='admin@korea.com' WHERE uid='admin';"   # admin@korea.com','admin' 가 플레이스 홀더
sql = "UPDATE users SET email=%s WHERE uid=%s;"
cur.execute(sql, (email, uid))  # 순서가 중요!! 작성한 순서대로 써야한다
conn.commit()  

In [30]:
# 각 아이디마다 for문 사용하여 email 추가하기
cur = conn.cursor()
sql = "UPDATE users SET email=%s WHERE uid=%s;"
for i in df.index:
    uid = df.uid[i]
    email = f'{uid}@email.com'
    cur.execute(sql, (email,uid))
conn.commit()  

- 데이터 조회

In [33]:
# is_deleted 가 0인 sql 생성
sql = """
    SELECT uid,uname,email,
    DATE_FORMAT(reg_date,'%Y-%m-%d %h:%i') AS reg_date
    FROM users WHERE is_deleted=0;  
"""

In [34]:
# 한 레코드 조회 
cur = conn.cursor()
cur.execute(sql)
result = cur.fetchone()    # 읽기만 하는거
result

('admin', '관리자', 'admin@korea.com', '2021-06-09 10:53')

In [35]:
# n개의 레코드 조회
cur.fetchmany(2)

(('djy', '대조영', 'djy@email.com', '2021-06-09 11:31'),
 ('eskim', '김은숙', 'eskim@email.com', '2021-06-09 11:31'))

In [36]:
# 모두 조회
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
for row in results:
    print(row)

('admin', '관리자', 'admin@korea.com', '2021-06-09 10:53')
('djy', '대조영', 'djy@email.com', '2021-06-09 11:31')
('eskim', '김은숙', 'eskim@email.com', '2021-06-09 11:31')
('gdhong', '홍길동', 'gdhong@email.com', '2021-06-09 11:31')
('jbpark', '박재범', 'jbpark@email.com', '2021-06-09 11:31')
('vincenzo', '빈센조', 'vincenzo@email.com', '2021-06-09 11:31')
('wjlee', '이우정', 'wjlee@email.com', '2021-06-09 11:31')


In [37]:
# 모두 조회한 뒤 데이터프레임으로 만들기
# 빈리스트 만들고 for loop안에서 데이터 넣어주고 맨마지막 list에 append 하는것!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 좀외우셈 모두 조회한 뒤 데이터프레임으로 만들기
# 빈리스트 만들고 for loop안에서 데이터 넣어주고 맨마지막 list에 append 하는것!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! 좀외우셈

uids, unames, emails, regs = [],[],[],[]
for row in results:
    uids.append(row[0])
        unames.append(row[1])
            emails.append(row[2])
                regs.append(row[3])
                

uids, unames, emails, regs = [],[],[],[]
for row in results:
    uids.append(row[0])
    unames.append(row[1])
    emails.append(row[2])
    regs.append(row[3])

df2 = pd.DataFrame({
    'uid': uids, 'uname': unames, 'email': emails, 'reg': regs
})
df2

,uid,uname,email,reg
0,admin,관리자,admin@korea.com,2021-06-09 10:53
1,djy,대조영,djy@email.com,2021-06-09 11:31
2,eskim,김은숙,eskim@email.com,2021-06-09 11:31
3,gdhong,홍길동,gdhong@email.com,2021-06-09 11:31
4,jbpark,박재범,jbpark@email.com,2021-06-09 11:31
5,vincenzo,빈센조,vincenzo@email.com,2021-06-09 11:31
6,wjlee,이우정,wjlee@email.com,2021-06-09 11:31


- 데이터 검색

In [39]:
# uid 하나만 뽑는다
keyword = '김'
sql = """
    SELECT uid                              
    FROM users
    WHERE is_deleted=0 and uname like %s;
    """

In [40]:
# 결과가 하나라도 튜플로 데이터를 반환  / fetchone 사용 결과가 하나라도 튜플로 데이터를 반환  / fetchone 사용
cur = conn.cursor()
cur.execute(sql, keyword+'%')   # keyword에 와일드캐릭터를 붙여줘야함, keyword는 '김'으로만 되어있기때문?
result = cur.fetchone()
result      # ex) uid 하나만 뽑았는데도 결과가 ('eskim',) 튜플로 나옴
cur = conn.cursor()
cur.execute(sql, keyword+'%')   # keyword에 와일드캐릭터를 붙여줘야함, keyword는 '김'으로만 되어있기때문?
result = cur.fetchone()
result      # ex) uid 하나만 뽑았는데도 결과가 ('eskim',) 튜플로 나옴

('eskim',)

In [41]:
result[0]

'eskim'

In [42]:
# fetchall 을 사용하면 튜플, 튜플
cur = conn.cursor()
cur.execute(sql, keyword+'%') 
result = cur.fetchall()
result     

(('eskim',),)

In [43]:
result[0][0]

'eskim'

In [44]:
# 다끝났으면 마지막 마무리로 써줘야함
cur.close()
conn.close()